In [106]:
from time import sleep
from tqdm import tqdm

import numpy as np
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials

from keys import GOOG_API_KEY

In [107]:
## Load dataframes
# Polling averages
pres = pd.read_csv('averages/presidential_gen_approval.csv')
pres_issue = pd.read_csv('averages/presidential_issue_approvals.csv')
pres_rv = pd.read_csv('averages/presidential_RV_approval.csv')
generic_ballot = pd.read_csv('averages/generic_ballot.csv')

# Polls display tables
pres_polls = pd.read_csv('transformed_tables/gen_approval_polls.csv')
issue_polls = pd.read_csv('transformed_tables/issue_approval_polls.csv')
gb_polls = pd.read_csv('transformed_tables/generic_ballot_polls_disp.csv')

In [108]:
# Define the scope
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# Authenticate with the service account
credentials = Credentials.from_service_account_file(
    GOOG_API_KEY,
    scopes=scopes
)

# Authorize gspread
client = gspread.authorize(credentials)

## Polling Averages

In [109]:
spreadsheet = client.open("snoutcounter-data")
worksheet = spreadsheet.worksheet("approval-stats")
worksheet.clear()

set_with_dataframe(worksheet, pres.reset_index())

In [110]:
worksheet = spreadsheet.worksheet("approval_stats_RV")
worksheet.clear()

set_with_dataframe(worksheet, pres_rv.reset_index())

In [111]:
issues = ['economy', 'immigration', 'inflation', 'trade_tariffs', 'foreign_policy', 'healthcare', 'crime']
issue_suffixes = ['econ', 'imm', 'infl', 'trade', 'fp', 'healthcare', 'crime']

for iss, suf in tqdm(zip(issues, issue_suffixes)):
    worksheet = spreadsheet.worksheet(f"approval_stats_{suf}")
    worksheet.clear()

    issue_df = pres_issue[pres_issue['issue'] == iss]
    set_with_dataframe(worksheet, issue_df.reset_index())

7it [00:06,  1.10it/s]


In [112]:
## Net issue approval
net_issues = pd.pivot_table(pres_issue[pres_issue['issue'].isin(issues)], 
                            values=['net', 'net_lower_ci', 'net_upper_ci'], index=['end_date'], columns=['issue'],
                           aggfunc="first").reset_index()
net_issues.columns = ['_'.join(col).strip() for col in net_issues.columns.values]
net_issues.head()

,end_date_,net_crime,net_economy,net_foreign_policy,net_healthcare,net_immigration,net_inflation,net_trade_tariffs,net_lower_ci_crime,net_lower_ci_economy,...,net_lower_ci_immigration,net_lower_ci_inflation,net_lower_ci_trade_tariffs,net_upper_ci_crime,net_upper_ci_economy,net_upper_ci_foreign_policy,net_upper_ci_healthcare,net_upper_ci_immigration,net_upper_ci_inflation,net_upper_ci_trade_tariffs
0,2025-01-21,0.0,7.274374,3.274374,0.000000,9.274374,-4.725626,-0.725626,0.0,7.274374,...,9.274374,-4.725626,-0.725626,0.0,7.274374,3.274374,0.000000,9.274374,-4.725626,-0.725626
1,2025-01-22,0.0,7.274374,3.274374,0.000000,9.274374,-4.725626,-0.725626,0.0,7.274374,...,9.274374,-4.725626,-0.725626,0.0,7.274374,3.274374,0.000000,9.274374,-4.725626,-0.725626
2,2025-01-23,0.0,0.893578,3.274374,-15.665098,1.466023,-4.725626,-0.725626,0.0,-7.025983,...,-8.225358,-4.725626,-0.725626,0.0,8.813140,3.274374,-15.665098,11.157403,-4.725626,-0.725626
3,2025-01-24,0.0,0.893578,3.274374,-15.665098,1.466023,-4.725626,-0.725626,0.0,-7.025983,...,-8.225358,-4.725626,-0.725626,0.0,8.813140,3.274374,-15.665098,11.157403,-4.725626,-0.725626
4,2025-01-25,0.0,0.893578,3.274374,-15.665098,1.466023,-4.725626,-0.725626,0.0,-7.025983,...,-8.225358,-4.725626,-0.725626,0.0,8.813140,3.274374,-15.665098,11.157403,-4.725626,-0.725626


In [113]:
worksheet = spreadsheet.worksheet("net_approval_issues")
worksheet.clear()

set_with_dataframe(worksheet, net_issues.reset_index())

In [114]:
worksheet = spreadsheet.worksheet("generic_ballot_stats")
worksheet.clear()

set_with_dataframe(worksheet, generic_ballot.reset_index())

## Polls Display Tables

In [115]:
pres_polls['sponsors'] = '^Sponsor: ' + pres_polls['sponsors'] + '^'
pres_polls['sponsors'] = pres_polls['sponsors'].fillna('')
pres_polls['partisan'] = pres_polls['partisan'].replace({
    'DEM': ' (D)',
    'REP': ' (R)',
    'WFP': ' (WF)'
}).fillna('')
pres_polls = pres_polls.sort_values(['total_weight'], ascending=False)
pres_polls = pres_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net'}, axis='columns')
pres_polls['Sample'] = pres_polls['sample_size'].astype(int).astype(str) + ' ' + pres_polls['population']
pres_polls['Start Date'] = pd.to_datetime(pres_polls['Start Date']).astype(str)
pres_polls['End Date'] = pres_polls['End Date'].astype(str)
pres_polls = pres_polls.reset_index()
pres_polls['Pollster'] = ('<a href="' + pres_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          pres_polls['Pollster'] + pres_polls['partisan'] + '</u></a>' + pres_polls['sponsors'])
pres_polls = pres_polls[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
pres_polls

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
0,"<a href=""https://napolitannews.org/jobapproval...",2026-01-28,2026-02-05,3000 RV,1.318669e-01,46.0,53.0,-7.0,-17.397240
1,"<a href=""https://poll.qu.edu/poll-release?rele...",2026-01-29,2026-02-02,1191 RV,6.983857e-02,37.0,56.0,-19.0,-15.575903
2,"<a href=""https://johnzogbystrategies.com/the-p...",2026-02-04,2026-02-05,1000 LV,6.699993e-02,44.0,54.0,-10.0,-14.157134
3,"<a href=""https://www.pewresearch.org/politics/...",2026-01-20,2026-01-26,8512 A,6.223907e-02,37.0,61.0,-24.0,-18.274227
4,"<a href=""https://maristpoll.marist.edu/polls/t...",2026-01-27,2026-01-30,1462 A,5.904921e-02,39.0,56.0,-17.0,-15.335597
...,...,...,...,...,...,...,...,...,...
561,"<a href=""https://mclaughlinonline.com/pols/wp-...",2025-01-22,2025-01-27,1000 LV,1.236247e-20,52.0,43.0,9.0,4.741007
562,"<a href=""https://substack.com/inbox/post/15539...",2025-01-21,2025-01-21,742 A,9.553306e-21,49.0,36.0,13.0,8.249316
563,"<a href=""https://www.ipsos.com/en-us/reuters-i...",2025-01-20,2025-01-21,1077 A,9.379163e-21,47.0,41.0,6.0,9.622506
564,"<a href=""https://quantusinsights.org/f/februar...",2025-02-01,2025-02-03,1000 RV,9.321950e-21,51.9,45.3,6.6,-0.506709


In [116]:
worksheet = spreadsheet.worksheet("polls_display_table")
worksheet.clear()

set_with_dataframe(worksheet, pres_polls)

In [117]:
issue_polls['sponsor'] = '^Sponsor: ' + issue_polls['sponsor'] + '^'
issue_polls['sponsor'] = issue_polls['sponsor'].fillna('')
issue_polls = issue_polls.sort_values(['issue', 'total_weight'], ascending=False)
issue_polls = issue_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net',
    'issue':'Issue'}, axis='columns')
issue_polls['Sample'] = issue_polls['sample_size'].astype(int).astype(str) + ' ' + issue_polls['population']
issue_polls['Start Date'] = pd.to_datetime(issue_polls['Start Date']).astype(str)
issue_polls['End Date'] = issue_polls['End Date'].astype(str)
issue_polls = issue_polls.reset_index()
issue_polls['Pollster'] = ('<a href="' + issue_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          issue_polls['Pollster'] + '</u></a>' + issue_polls['sponsor'])
issue_polls = issue_polls[issue_polls['Issue'].isin(issues)]
issue_polls['Issue'] = issue_polls['Issue'].replace({
    'economy': 'Economy',
    'immigration': 'Immigration',
    'foreign_policy': 'Foreign policy',
    'inflation': 'Inflation/prices',
    'trade_tariffs': 'Trade/tariffs',
    'healthcare': 'Healthcare',
    'crime': 'Crime'
})
issue_polls = issue_polls[['Pollster', 'Start Date', 'End Date', 'Issue', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
issue_polls

,Pollster,Start Date,End Date,Issue,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
0,"<a href=""https://johnzogbystrategies.com/the-p...",2026-02-04,2026-02-05,Trade/tariffs,1000 LV,1.857518e-02,40.0,55.0,-15.0,-23.088549
1,"<a href=""https://law.marquette.edu/assets/comm...",2026-01-21,2026-01-28,Trade/tariffs,1003 A,1.825035e-02,37.0,62.0,-25.0,-19.722469
2,"<a href=""https://harvardharrispoll.com/wp-cont...",2026-01-28,2026-01-29,Trade/tariffs,2000 RV,1.377677e-02,39.0,54.0,-15.0,-22.895359
3,"<a href=""https://static.foxnews.com/foxnews.co...",2026-01-23,2026-01-26,Trade/tariffs,1005 RV,1.149142e-02,37.0,63.0,-26.0,-23.739840
4,"<a href=""https://pro.morningconsult.com/tracke...",2026-01-30,2026-02-01,Trade/tariffs,2201 RV,1.028569e-02,44.0,47.0,-3.0,-16.861307
...,...,...,...,...,...,...,...,...,...,...
1125,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-23,2025-02-25,Crime,1604 A,1.229130e-14,47.0,37.0,10.0,7.377607
1126,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-16,2025-02-18,Crime,1603 A,7.114018e-15,48.0,37.0,11.0,8.377607
1127,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-09,2025-02-11,Crime,1595 A,4.124032e-15,45.0,41.0,4.0,1.377607
1128,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-02,2025-02-04,Crime,1604 A,2.416632e-15,48.0,35.0,13.0,10.377607


In [118]:
worksheet = spreadsheet.worksheet("polls_display_issues")
worksheet.clear()

set_with_dataframe(worksheet, issue_polls)

In [119]:
gb_polls['sponsors'] = '^Sponsor: ' + gb_polls['sponsors'] + '^'
gb_polls['partisan'] = gb_polls['partisan'].replace({
    'DEM': ' (D)',
    'REP': ' (R)',
    'WFP': ' (WF)'
}).fillna('')
gb_polls['sponsors'] = gb_polls['sponsors'].fillna('')
gb_polls = gb_polls.sort_values(['total_weight'], ascending=False)
gb_polls = gb_polls.rename({
    'pollster':'Pollster', 'dem':'Democrats', 'rep':'Republicans', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net'}, axis='columns')
gb_polls['Sample'] = gb_polls['sample_size'].astype(int).astype(str) + ' ' + gb_polls['population']
gb_polls['Start Date'] = pd.to_datetime(gb_polls['Start Date']).astype(str)
gb_polls['End Date'] = gb_polls['End Date'].astype(str)
gb_polls = gb_polls.reset_index()
gb_polls['Pollster'] = ('<a href="' + gb_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          gb_polls['Pollster'] + gb_polls['partisan'] + '</u></a>' + gb_polls['sponsors'])
gb_polls = gb_polls[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Democrats', 'Republicans', 'Net', 'Adjusted Net']]
gb_polls

,Pollster,Start Date,End Date,Sample,Weight,Democrats,Republicans,Net,Adjusted Net
0,"<a href=""https://www.cygn.al/polls/national-po...",2026-01-27,2026-01-28,1004 LV,9.592110e-02,48.0,44.0,-4.0,-5.497576
1,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2026-01-30,2026-02-02,1504 RV,8.886323e-02,44.0,40.0,-4.0,-3.951474
2,"<a href=""https://law.marquette.edu/poll/2026/0...",2026-01-21,2026-01-28,577 LV,8.462970e-02,52.0,45.0,-7.0,-6.365176
3,"<a href=""https://echelonin.wpenginepowered.com...",2026-01-22,2026-01-26,1029 LV,7.322004e-02,49.0,44.0,-5.0,-7.248037
4,"<a href=""https://static.foxnews.com/foxnews.co...",2026-01-23,2026-01-26,1005 RV,7.209336e-02,52.0,46.0,-6.0,-3.984894
...,...,...,...,...,...,...,...,...,...
287,"<a href=""https://www.cygn.al/voters-back-kingm...",2025-01-09,2025-01-12,1500 LV,2.148073e-20,44.6,46.8,2.2,1.062648
288,"<a href=""https://mclaughlinonline.com/pols/wp-...",2025-01-22,2025-01-27,1000 LV,1.574677e-20,40.0,48.0,8.0,2.605646
289,"<a href=""https://quantusinsights.org/f/trump%E...",2025-01-22,2025-01-23,1000 RV,8.060939e-21,45.0,48.0,3.0,0.973194
290,"<a href=""https://www.cygn.al/poll-musk-most-ef...",2024-12-09,2024-12-11,1500 LV,6.226230e-22,44.6,48.4,3.8,2.662648


In [120]:
worksheet = spreadsheet.worksheet("generic_ballot_polls_display_table")
worksheet.clear()

set_with_dataframe(worksheet, gb_polls)